**Getting and cleaning data from postgress**

In [6]:
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [7]:
# Postgres connection

connection_args = {
    'host': '',  
    'dbname': '',
    'user': '',
    'port':          
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)

In [8]:
# Query for pulling necessary data from postgress database

query = '''
select * from 
	(
	select 
		case 
			when review_scores_value = 10 then 1
			else 0 end as review_10,
        case
			when review_scores_value in (9,10) then 1
			else 0 end as review_9or10,
        review_scores_value,
        case
        	when host_is_superhost = 't' then 1
        	else 0 end as superhost,
		(last_scraped - host_since) as host_days,
		(last_scraped - host_since) as host_days,   
		case
			when host_response_rate = 'N/A' then null 
			else regexp_replace(host_response_rate , '%', '','g') end as response_rate,
		host_listings_count,
		host_verifications,
		case 
			when host_has_profile_pic = 't' then 1
			else 0 end as host_pic,
		case 
			when host_identity_verified = 't' then 1
			else 0 end as id_verified,
		neighbourhood_cleansed,
		neighbourhood_group_cleansed as borough,
		case
			when length(regexp_replace(zipcode, '/d', '','g')) = 5
				then cast(regexp_replace(zipcode, '/d', '','g') as int)
				else null end as zipcode1,
		property_type,
		room_type,
		accommodates,
		bathrooms,
		bedrooms,
		beds,
		case
			when bed_type = 'Real Bed' then 1
			else 0 end as real_bed,
		amenities,
		cast(regexp_replace(price, '\$|\.00|,', '','g') as int) as price,
		cast(regexp_replace(security_deposit, '\$|\.00|,', '','g') as int) as sec_dep,
		cast(regexp_replace(cleaning_fee, '\$|\.00|,', '','g') as int) as cleaning_fee,
		cast(regexp_replace(extra_people , '\$|\.00|,', '','g') as int) as extra_people,
		guests_included,
		minimum_nights,
		maximum_nights,
		number_of_reviews,
		case 
			when instant_bookable = 't' then 1
			else 0 end as instant_bookable,
		case 
			when is_business_travel_ready ='t' then 1
			else 0 end as business_travel_ready,
		cancellation_policy,
        latitude as lat,
        longitude as lon
	from listings
	where review_scores_value != 0
	) main_table
	join
	(
	select
		ZIP as zipcode2,
		count(*) as galleries from Galleries group by ZIP
	) gal_table
	on main_table.zipcode1 = gal_table.zipcode2
	join
	(
	select
		case 
			when ZIP_CODE > 0 then ZIP_CODE
			else null end as zipcode3,
		sum(case when result in ('Active Rat Signs', 'Bait applied','Cleanup done','Problem Conditions')
			then 1 else 0 end) as rats
	from Rodents
	group by ZIP_CODE
	) rat_table
	on main_table.zipcode1 = rat_table.zipcode3
;
'''

airbnb = pd_sql.read_sql(query, connection)


In [9]:
# Cleaning dataframe

airbnb = airbnb.drop(['zipcode1','zipcode2','zipcode3'], axis=1)
airbnb.review_scores_value = airbnb.review_scores_value.astype(int)
airbnb.host_verifications = airbnb.host_verifications.apply(lambda x: x[1:-1].split(','))
airbnb.host_verifications = airbnb.host_verifications.apply(lambda x: len(x))
airbnb.amenities = airbnb.amenities.apply(lambda x: x[1:-1].split(','))
airbnb.amenities = airbnb.amenities.apply(lambda x: len(x))
airbnb = airbnb.dropna().reset_index()
airbnb.response_rate = airbnb.response_rate.astype(int)
airbnb = airbnb.drop('index', axis=1)

In [10]:
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15174 entries, 0 to 15173
Data columns (total 36 columns):
review_10                 15174 non-null int64
review_9or10              15174 non-null int64
review_scores_value       15174 non-null int64
superhost                 15174 non-null int64
host_days                 15174 non-null float64
host_days                 15174 non-null float64
response_rate             15174 non-null int64
host_listings_count       15174 non-null float64
host_verifications        15174 non-null int64
host_pic                  15174 non-null int64
id_verified               15174 non-null int64
neighbourhood_cleansed    15174 non-null object
borough                   15174 non-null object
property_type             15174 non-null object
room_type                 15174 non-null object
accommodates              15174 non-null int64
bathrooms                 15174 non-null float64
bedrooms                  15174 non-null float64
beds                      15174

In [11]:
# Export data
airbnb.to_csv('airbnb_FINAL.csv')